# Value Iteration

Vamos a ver como la programación dinamica se puede utilizar para derivar políticas óptimas para algunos ambientes. En este caso particular vamos a utilizar dos ambientes definidos en el libro de Sutton y Barto. 

Es recomendable leer el capítulo 4 del libro para entender a fondo todos los conceptos.

## A entregar:

- Notebook con el algoritmo de value iteration implementado (hecho en clase)
- Comentarios sobre funciones implementadas en clase, que hacen y que es cada parametro.
- Reporte con funciones de valor para diferentes tamaños de ambiente.
- Reporte con graficas sobre el numero de iteraciones del algoritmo para diferentes tamaños de ambiente.


## Definicion del ambiente

En este caso vamos a usar como ambiente algo muy sencillo, Gridworld (del libro de Sutton y Barto). Basicamente se trata de una grilla rectangular de posibles posiciones donde los estados terminales se encuentran en la esquina superior izquierda ([0,0] en coordenadas) y en la esquina inferior derecha ([largo - 1, ancho - 1] en coordenadas). 

El objetivo es obtener la función de valor óptima para el entorno en el que estamos.

Comenzamos definiendo el ambiente como una clase Python abajo, esto nos va a permitir crear varios ambientes de prueba para probar nuestro algoritmo y encontrar la función optima para todos.

In [19]:
# Necessary imports
import io
import sys
import numpy as np
#from gym.envs.toy_text import discrete
from DiscreteEnv import DiscreteEnv

# Possible Actions
UP = 0
RIGHT = 1
DOWN = 2
LEFT = 3

In [20]:
class GridworldEnv(DiscreteEnv):
    """
    ****
    Forked from: https://github.com/dennybritz/reinforcement-learning/blob/master/lib/envs/gridworld.py
    ****
    Grid World environment from Sutton's Reinforcement Learning book chapter 4.
    You are an agent on an MxN grid and your goal is to reach the terminal
    state at the top left or the bottom right corner.

    For example, a 4x4 grid looks as follows:

    T  o  o  o
    o  x  o  o
    o  o  o  o
    o  o  o  T

    x is your position and T are the two terminal states.

    You can take actions in each direction (UP=0, RIGHT=1, DOWN=2, LEFT=3).
    Actions going off the edge leave you in your current state.
    You receive a reward of -1 at each step until you reach a terminal state.
    """

    def __init__(self, shape=[4,4]):
        if not isinstance(shape, (list, tuple)) or not len(shape) == 2:
            raise ValueError('Shape argument must be a list/tuple of length 2')

        self.shape = shape

        number_states = np.prod(shape)
        number_actions = 4
        
        self.number_actions = number_actions
        self.number_states = number_states

        MAX_Y = shape[0]
        MAX_X = shape[1]

        transition_probabilities = {}
        grid = np.arange(number_states).reshape(shape)
        it = np.nditer(grid, flags=['multi_index'])

        while not it.finished:
            s = it.iterindex
            y, x = it.multi_index

            transition_probabilities[s] = {a : [] for a in range(number_actions)}

            is_done = lambda s: s == 0 or s == (number_states - 1)
            reward = 0.0 if is_done(s) else -1.0

            # We're stuck in a terminal state
            if is_done(s):
                transition_probabilities[s][UP] = [(1.0, s, reward, True)]
                transition_probabilities[s][RIGHT] = [(1.0, s, reward, True)]
                transition_probabilities[s][DOWN] = [(1.0, s, reward, True)]
                transition_probabilities[s][LEFT] = [(1.0, s, reward, True)]
            # Not a terminal state
            else:
                ns_up = s if y == 0 else s - MAX_X
                ns_right = s if x == (MAX_X - 1) else s + 1
                ns_down = s if y == (MAX_Y - 1) else s + MAX_X
                ns_left = s if x == 0 else s - 1
                transition_probabilities[s][UP] = [(1.0, ns_up, reward, is_done(ns_up))]
                transition_probabilities[s][RIGHT] = [(1.0, ns_right, reward, is_done(ns_right))]
                transition_probabilities[s][DOWN] = [(1.0, ns_down, reward, is_done(ns_down))]
                transition_probabilities[s][LEFT] = [(1.0, ns_left, reward, is_done(ns_left))]

            it.iternext()

        # Initial state distribution is uniform
        isd = np.ones(number_states) / number_states

        # We expose the model of the environment for educational purposes
        # This should not be used in any model-free learning algorithm
        self.transition_probabilities = transition_probabilities

        super(GridworldEnv, self).__init__(number_states, number_actions, transition_probabilities, isd)

## Inicializando una nueva instancia del ambiente

En la celda de abajo vamos a crear una nueva instancia del ambiente definido arriba. En este caso, y por ser un ejemplo, conocemos la funcion de transición del ambiente (llamada `transition_probabilities`). Esta función nos dice, para cada estado, el resultado de tomar cada accion posible.

*** 

Esta funcion esta representada por tuplas de la forma: `(Probabilidad, estado_siguiente, recompensa, estado_final)`. 

- Al ser un ambiente deterministico las **probabilidades** son siempre 1 (al tomar una accion tenemos 100% de chances de caer en el siguiente estado, otros casos una acción podría tener una distribuccion de probabilidades sobre todos los estados posibles).

- Los **estados_siguientes** apuntan a dónde mirar en la funcion de transición.

- Las **recompensas** como se mencionó arriba son siempre -1 por cada movimiento, excepto cuando estamos en un estado final.

- El **estad_final** es un flag booleano (Verdadero o Falso) que nos dice si el estado al que nos movemos es final o no.

***

Por ejemplo: si estamos en el estado 0, tomar cualquier accion tiene recompensa 0, con probabilidad 100% (es un ambiente determinístico) y 

In [29]:
grid = GridworldEnv()   # Create a new gridworld object
grid.transition_probabilities

# Recordar: 
# UP = 0
# RIGHT = 1
# DOWN = 2
# LEFT = 3

{0: {0: [(1.0, 0, 0.0, True)],
  1: [(1.0, 0, 0.0, True)],
  2: [(1.0, 0, 0.0, True)],
  3: [(1.0, 0, 0.0, True)]},
 1: {0: [(1.0, 1, -1.0, False)],
  1: [(1.0, 2, -1.0, False)],
  2: [(1.0, 5, -1.0, False)],
  3: [(1.0, 0, -1.0, True)]},
 2: {0: [(1.0, 2, -1.0, False)],
  1: [(1.0, 3, -1.0, False)],
  2: [(1.0, 6, -1.0, False)],
  3: [(1.0, 1, -1.0, False)]},
 3: {0: [(1.0, 3, -1.0, False)],
  1: [(1.0, 3, -1.0, False)],
  2: [(1.0, 7, -1.0, False)],
  3: [(1.0, 2, -1.0, False)]},
 4: {0: [(1.0, 0, -1.0, True)],
  1: [(1.0, 5, -1.0, False)],
  2: [(1.0, 8, -1.0, False)],
  3: [(1.0, 4, -1.0, False)]},
 5: {0: [(1.0, 1, -1.0, False)],
  1: [(1.0, 6, -1.0, False)],
  2: [(1.0, 9, -1.0, False)],
  3: [(1.0, 4, -1.0, False)]},
 6: {0: [(1.0, 2, -1.0, False)],
  1: [(1.0, 7, -1.0, False)],
  2: [(1.0, 10, -1.0, False)],
  3: [(1.0, 5, -1.0, False)]},
 7: {0: [(1.0, 3, -1.0, False)],
  1: [(1.0, 7, -1.0, False)],
  2: [(1.0, 11, -1.0, False)],
  3: [(1.0, 6, -1.0, False)]},
 8: {0: [(1.0, 4

## Recordando Value Iteration del libro de Sutton y Barto:

![Image](https://i.imgur.com/PBtFBCG.png)


Este es el algoritmo que vamos a implementar en las celdas siguientes.

La función de valor del ambiente anterior debería ser:
[ 0. -1. -2. -3. -1. -2. -3. -2. -2. -3. -2. -1. -3. -2. -1.  0.]

In [ ]:
def compute_action_values(env, state, V, discount_factor=1.0):
    """
    Helper function to calculate the value for all actions in a given state.
    
    Args:
        env: openAI.gym environment
        state: The state to consider (int)
        V: The value to use as an estimator, Vector of length env.number_states
    
    Returns:
        A vector of length env.number_actions containing the expected value of each action.
    """   

    
    pass

In [ ]:
def value_iteration(env, theta=0.0001, discount_factor=1.0):
    """
    Value Iteration Algorithm.
    
    Args:
        env: OpenAI env. env.transition_probabilities represents the transition probabilities of the environment.
            env.transition_probabilities[s][a] is a list of transition tuples (prob, next_state, reward, done).
            env.number_states is a number of states in the environment. 
            env.number_actions is a number of actions in the environment.
        theta: We stop evaluation once our value function change is less than theta for all states.
        discount_factor: Gamma discount factor.
        
    Returns:
        A tuple (policy, V) of the optimal policy and the optimal value function.
    """
    pass

In [ ]:
env = GridworldEnv()
policy, v, count = value_iteration(env)   # get the count as well

print(f"For env size 4 the number of iterations was {count}")

In [ ]:
print("Policy Probability Distribution:")
print(policy)
print("")

In [ ]:
print("Reshaped Grid Policy (0=up, 1=right, 2=down, 3=left):")
print(np.reshape(np.argmax(policy, axis=1), env.shape))
print("")

In [ ]:
print("Value Function:")
print(v)
print("")

In [ ]:
import matplotlib.pyplot as plt


env_sizes = range(1, 50)  # Explore some size range  
counts = []
value_functions = []

for size in env_sizes:
    env = GridworldEnv(shape=[size, size])
    policy, v, count = value_iteration(env)
    counts.append(count)
    value_functions.append(v)

    
print("Algunas funciones de valor:")
print(f"Ambiente 5x5: {value_functions[5]}")
print(f"Ambiente 25x25: {value_functions[25]}")


plt.plot(env_sizes, counts)
plt.xlabel("Tamaño de ambiente")
plt.ylabel("Iteraciones")
plt.title("Iteraciones para distintos tamaños de ambiente")
plt.show()